In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("awsaf49/food-recognition-2022-dataset")

# print("Path to dataset files:", path)

In [2]:
image_data_path = r'/kaggle/input/food-recognition-2022-dataset/public_validation_set_2.0/images/'
val_annot_path = r'/kaggle/input/wholedataset/xyz/xyz/val.json'

In [3]:
import json
import pandas as pd
from collections import OrderedDict

food = json.load(open(val_annot_path))
categories = food["categories"]
class_names = [category["name"] for category in categories]
category_ids = [category["id"] for category in categories]

no_images_per_category = {}
image_data = [] 

for n, cat_id in enumerate(category_ids):
    img_ids = [ann['image_id'] for ann in food['annotations'] if ann['category_id'] == cat_id]
    label = class_names[n]
    no_images_per_category[label] = len(img_ids)

    for img_id in img_ids:
        img_info = next((img for img in food['images'] if img['id'] == img_id), None)
        if img_info:
            image_data.append({
                'file_name': img_info['file_name'],
                'height': img_info['height'],
                'width': img_info['width'],
                'id': img_info['id'],
                'category': label
            })

df_image_info = pd.DataFrame(image_data)

no_images_per_category = OrderedDict(sorted(no_images_per_category.items(), key=lambda x: -1 * x[1]))


top_30_categories = []
i = 0
for k, v in no_images_per_category.items():
    print(k, v)
    top_30_categories.append((k, v))
    i += 1
    if i >= 30:
        break

water 77
salad-leaf-salad-green 62
tomato-raw 60
butter 41
bread-white 39
bread-wholemeal 36
coffee-with-caffeine 27
carrot-raw 27
cucumber 26
rice 24
bread-whole-wheat 23
jam 21
egg 21
bell-pepper-red-raw 19
apple 18
cheese 17
white-coffee-with-caffeine 17
wine-white 16
espresso-with-caffeine 14
pasta 13
mixed-vegetables 13
blueberries 13
onion 13
hummus 13
braided-white-loaf 13
hard-cheese 12
bread 12
mixed-salad-chopped-without-sauce 12
strawberries 12
salmon-smoked 11


In [4]:
df_image_info.to_csv("/kaggle/working/val_data.csv")

In [6]:
val_data = pd.read_csv("/kaggle/input/valdatacsv/val_data.csv")
val_data.columns, val_data.isna().sum()

(Index(['id', 'file_name', 'height', 'width', 'category'], dtype='object'),
 id           0
 file_name    0
 height       0
 width        0
 category     0
 dtype: int64)

In [7]:
len(val_data['category'].unique())

406

In [8]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
import os
import joblib
import pickle
from fastprogress import master_bar, progress_bar
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
import numpy as np
import pandas as pd
from PIL import Image

print(torchvision.__version__)

0.19.0


In [10]:
food_dataset = r'/kaggle/input/valdatacsv/val_data.csv'

In [11]:
# train_images_path = r'/kaggle/input/food-recognition-2022-dataset/public_training_set_release_2.0/images'
test_images_path = r'/kaggle/input/food-recognition-2022-dataset/public_test_release_2.0/images'
val_images_path = r'/kaggle/input/food-recognition-2022-dataset/public_validation_set_2.0/images'

In [13]:
IMG_WIDTH=224
IMG_HEIGHT=224
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)
validation_ratio = 0.2
batch_size = 64
width_shift_range=0.2,

height_shift_range=0.6,
zoom_range=0.3, 
shear_range=0.4, 
rotation_range=10

In [15]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.labels_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        self.label_mapping = {label: idx for idx, label in enumerate(self.labels_frame['category'].unique())}
        
    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.labels_frame.iloc[idx]['file_name'])
        image = Image.open(img_name).convert("RGB")
        label_str = self.labels_frame.iloc[idx]['category']
        label = self.label_mapping[label_str]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label)

In [17]:
csv_file = food_dataset
root_dir = '/kaggle/input/food-recognition-2022-dataset/public_validation_set_2.0/images/'
dataset = CustomImageDataset(csv_file=csv_file, root_dir=root_dir)

print(f'Total images: {len(dataset)}')
img, label = dataset[0] 
print(f'Image shape: {img.size}, Label: {label}')

Total images: 1830
Image shape: (870, 870), Label: 0


In [18]:
transform = transforms.Compose([
    transforms.Resize((IMG_WIDTH, IMG_HEIGHT)),
    transforms.RandomAffine(
        degrees = rotation_range,
        translate = (width_shift_range[0], height_shift_range[0]),
        scale = (1 - zoom_range[0], 1 + zoom_range[0]),
        shear = shear_range[0]
    ),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [19]:
dataset = CustomImageDataset(csv_file=csv_file, root_dir=root_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [21]:
validation_data = 0.2
train_size = int((1 - validation_data) * 1830)
test_size = 1830 - train_size

train_size,test_size

(1464, 366)

In [22]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.model = torchvision.models.resnet50(pretrained=True) 
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)  

    def forward(self, x):
        return self.model(x)

In [23]:
# class MyModel3(nn.Module):
#     def __init__(self):
#         super(MyModel3, self).__init__()
#         no_Of_Filters = 32
#         size_of_Filter = (2, 2)
#         size_of_pool = (2, 2)
#         no_Of_Nodes = 250

#         self.conv1 = nn.Conv2d(3, no_Of_Filters, kernel_size=size_of_Filter, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=size_of_pool)
#         self.batch_norm1 = nn.BatchNorm2d(no_Of_Filters)

#         self.conv2 = nn.Conv2d(no_Of_Filters, no_Of_Filters*2, kernel_size=size_of_Filter, padding=1)
#         self.conv3 = nn.Conv2d(no_Of_Filters*2, no_Of_Filters*4, kernel_size=size_of_Filter, padding=1)
#         self.batch_norm2 = nn.BatchNorm2d(no_Of_Filters)

#         self.dropout = nn.Dropout(0.4)
#         self.flatten = nn.Flatten()
#         self.fc1 = nn.Linear(no_Of_Filters * 8 * 8, no_Of_Nodes)
#         self.fc2 = nn.Linear(no_Of_Nodes, 498)

#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = self.pool(x)
#         x = self.batch_norm1(x)

#         x = F.relu(self.conv2(x))
#         x = self.pool(x)

#         x = F.relu(self.conv3(x))
#         x = self.pool(x)
#         x = self.dropout(x)

#         x = self.flatten(x)
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)

#         return x

# model2 = MyModel3()
# model2.to(device)


In [24]:
num_classes = dataset.labels_frame['category'].nunique()  

model = SimpleCNN(num_classes=num_classes)
# model = model2
model = model.to('cuda' if torch.cuda.is_available() else 'cpu') 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 173MB/s] 


In [25]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100):
    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')  
    for epoch in range(num_epochs):
        model.train() 
        running_loss = 0.0

        for images, labels in train_loader:
            images = images.to('cuda' if torch.cuda.is_available() else 'cpu')  
            labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')  

            optimizer.zero_grad()  
            outputs = model(images) 
            loss = criterion(outputs, labels)  
            loss.backward() 
            optimizer.step()  

            running_loss += loss.item() 

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in val_loader:
                images = images.to('cuda' if torch.cuda.is_available() else 'cpu')
                labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print(f'Validation Accuracy: {100 * correct / total:.2f}%')

train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100)

Epoch [1/100], Loss: 5.5837
Validation Accuracy: 12.02%
Epoch [2/100], Loss: 4.8018
Validation Accuracy: 12.02%
Epoch [3/100], Loss: 4.4533
Validation Accuracy: 12.30%
Epoch [4/100], Loss: 4.1153
Validation Accuracy: 12.84%
Epoch [5/100], Loss: 3.8112
Validation Accuracy: 13.93%
Epoch [6/100], Loss: 3.5032
Validation Accuracy: 15.30%
Epoch [7/100], Loss: 3.2047
Validation Accuracy: 13.93%
Epoch [8/100], Loss: 2.9770
Validation Accuracy: 13.39%
Epoch [9/100], Loss: 2.6975
Validation Accuracy: 14.48%
Epoch [10/100], Loss: 2.4934
Validation Accuracy: 12.84%
Epoch [11/100], Loss: 2.2557
Validation Accuracy: 13.39%
Epoch [12/100], Loss: 2.0777
Validation Accuracy: 12.57%
Epoch [13/100], Loss: 1.9277
Validation Accuracy: 13.39%
Epoch [14/100], Loss: 1.8003
Validation Accuracy: 13.11%
Epoch [15/100], Loss: 1.6865
Validation Accuracy: 12.57%
Epoch [16/100], Loss: 1.5578
Validation Accuracy: 11.20%
Epoch [17/100], Loss: 1.4934
Validation Accuracy: 14.21%
Epoch [18/100], Loss: 1.4350
Validation 

In [27]:
num_epochs = 100
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/100], Loss: 2.1292
Epoch [2/100], Loss: 2.0453
Epoch [3/100], Loss: 1.3786
Epoch [4/100], Loss: 1.2422
Epoch [5/100], Loss: 1.2219
Epoch [6/100], Loss: 0.8664
Epoch [7/100], Loss: 0.7911
Epoch [8/100], Loss: 1.3925
Epoch [9/100], Loss: 0.9634
Epoch [10/100], Loss: 0.8732
Epoch [11/100], Loss: 0.8929
Epoch [12/100], Loss: 1.0228
Epoch [13/100], Loss: 0.7585
Epoch [14/100], Loss: 1.0953
Epoch [15/100], Loss: 0.9322
Epoch [16/100], Loss: 0.7810
Epoch [17/100], Loss: 1.0033
Epoch [18/100], Loss: 1.0146
Epoch [19/100], Loss: 0.6216
Epoch [20/100], Loss: 0.8615
Epoch [21/100], Loss: 1.0239
Epoch [22/100], Loss: 0.8810
Epoch [23/100], Loss: 0.7376
Epoch [24/100], Loss: 1.1829
Epoch [25/100], Loss: 1.0403
Epoch [26/100], Loss: 0.6517
Epoch [27/100], Loss: 1.1428
Epoch [28/100], Loss: 0.8678
Epoch [29/100], Loss: 0.8200
Epoch [30/100], Loss: 0.8641
Epoch [31/100], Loss: 0.9103
Epoch [32/100], Loss: 1.2333
Epoch [33/100], Loss: 1.0544
Epoch [34/100], Loss: 0.8082
Epoch [35/100], Loss: 0

In [28]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(1830, 100 * correct / total))

Accuracy of the network on the 1830 train images: 60.51912568306011 %


In [32]:
with open("/kaggle/working/data.pkl", "wb") as file:
    pickle.dump(model, file)

In [ ]:
class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer5 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size=3)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
      
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
        
        out = self.conv_layer5(out)
        out = self.max_pool3(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [ ]:
custom_model = ConvNeuralNet(num_classes)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(custom_model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

total_step = len(train_loader)

In [ ]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = custom_model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(1830, 100 * correct / total))